In [13]:
from operator import itemgetter

from langchain.chat_models import ChatLiteLLM
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.vectorstores import FAISS

In [14]:
import os
from dotenv import load_dotenv

load_dotenv()
cohere_api_key= os.getenv('GOOGLE_API_KEY')
huggingfacehub_api_key= os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [15]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import FAISS

pdf_loader = DirectoryLoader('../../../../data/tenders', glob="**/*.pdf")
txt_loader = DirectoryLoader('../../../../data/tenders', glob="**/*.txt")
word_loader = DirectoryLoader('../../../../data/tenders', glob="**/*.docx")
csv_loader = DirectoryLoader('../../../../data/tenders', glob="**/*.csv")

In [17]:
from bs4 import BeautifulSoup as Soup
from langchain.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain.document_loaders import PlaywrightURLLoader

url = "https://spiderman.fandom.com/wiki"
recursive_url_loader = RecursiveUrlLoader(url=url, max_depth=5, extractor=lambda x: Soup(x, "html.parser").text)


loaders = [recursive_url_loader]
documents = []
for loader in loaders:
    documents.extend(loader.load())

print(f"Total number of documents: {len(documents)}")

/tmp/ipykernel_15085/1800906842.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  recursive_url_loader = RecursiveUrlLoader(url=url, max_depth=5, extractor=lambda x: Soup(x, "html.parser").text)


In [ ]:
db = FAISS.from_documents(docs, HuggingFaceEmbeddings())
retriever = db.as_retriever()

NameError: name 'docs' is not defined

In [ ]:


template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatLiteLLM()

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke("Who is the Green Goblin?")